In [4]:
from neo4j import GraphDatabase
from tqdm import tqdm_notebook as tqdm
import pandas as pd


In [5]:
# Connect to the neo4j server
url = "neo4j://localhost:7687"
username = "neo4j"
password = "dsaa5002"

driver = GraphDatabase.driver(url, auth=(username, password))

In [6]:
# Import data
news_with_sentiment = pd.read_excel(r"D:\Career\HKUST(GZ)\5002\dsaa5002_project\task1.xlsx")

In [7]:
# # Convert from string to list
news_with_sentiment['Explicit_Company'] = news_with_sentiment['Explicit_Company'].apply(lambda x: x[1:-1].split(', '))


In [8]:
def match_relationships(tx, start_company):
    query = (
        "MATCH (a:Company {name: $company_name})-[r]->(b) "
        "RETURN b.name as company, type(r) as relation"
    )
    result = tx.run(query, company_name=start_company)
    return [(record["company"], record["relation"]) for record in result]
    
opposite_impact_sentiment = ['compete', 'dispute']
same_impact_sentiment = ['cooperate', 'invest', 'same_industry', 'supply']

In [ ]:
Implicit_Positive_Company = []
Implicit_Negative_Company = []
with driver.session() as session:
    for index, row in tqdm(news_with_sentiment.iterrows(), total=news_with_sentiment.shape[0]):
        positive = []
        negative = []
        for start_company in row['Explicit_Company']:
            relationships = session.execute_read(match_relationships, start_company)
            for end_company, relation in relationships:
                if row['label'] == 0:
                    if relation in opposite_impact_sentiment:
                        negative.append(end_company)
                    elif relation in same_impact_sentiment:
                        positive.append(end_company)
                else:
                    if relation in same_impact_sentiment:
                        negative.append(end_company)
                    elif relation in opposite_impact_sentiment:
                        positive.append(end_company)                        
        Implicit_Positive_Company.append(positive)
        Implicit_Negative_Company.append(negative)

C:\Users\Bella\AppData\Local\Temp\ipykernel_12632\832600505.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm(news_with_sentiment.iterrows(), total=news_with_sentiment.shape[0]):


  0%|          | 0/496053 [00:00<?, ?it/s]

In [ ]:
# # only one company in 'Explicit_company'
# Implicit_Positive_Company = []
# Implicit_Negative_Company = []
# with driver.session() as session:
#     for index, row in tqdm(news_with_sentiment.iterrows(), total=news_with_sentiment.shape[0]):
#         start_company = row['Explicit_Company']
#         relationships = session.execute_read(match_relationships, start_company)
#         positive = []
#         negative = []
#         for end_company, relation in relationships:
#             if row['label'] == 0:
#                 if relation in opposite_impact_sentiment:
#                     negative.append(end_company)
#                 elif relation in same_impact_sentiment:
#                     positive.append(end_company)
#             else:
#                 if relation in same_impact_sentiment:
#                     negative.append(end_company)
#                 elif relation in opposite_impact_sentiment:
#                     positive.append(end_company)
#         Implicit_Positive_Company.append(positive)
#         Implicit_Negative_Company.append(negative)

In [ ]:
news_with_sentiment['Implicit_Positive_Company'] = Implicit_Positive_Company
news_with_sentiment['Implicit_Negative_Company'] = Implicit_Negative_Company

In [ ]:
news_with_sentiment

In [ ]:
def list_to_string(lst):
    # 将列表转换为字符串
    return ','.join(lst)
news_with_sentiment['Implicit_Positive_Company'] = news_with_sentiment['Implicit_Positive_Company'].apply(list_to_string)
news_with_sentiment['Implicit_Negative_Company'] = news_with_sentiment['Implicit_Negative_Company'].apply(list_to_string)

In [ ]:
news_with_sentiment.to_excel(r"D:\Career\HKUST(GZ)\5002\dsaa5002_project\task2.xlsx", index=False)